In [1]:
import cv2
import random
import pyttsx3
import mediapipe as mp
import time
import threading

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def get_winner(user_choice, computer_choice):
    if user_choice == computer_choice:
        return "It's a tie!"
    elif (
        (user_choice == "rock" and computer_choice == "scissors") or
        (user_choice == "paper" and computer_choice == "rock") or
        (user_choice == "scissors" and computer_choice == "paper")
    ):
        return "You win!"
    else:
        return "Computer wins!"

def get_hand_gesture(landmarks):
    index_finger_tip = landmarks.landmark[8]
    middle_finger_tip = landmarks.landmark[12]
    
    distance = ((middle_finger_tip.x - index_finger_tip.x) ** 2 +
                (middle_finger_tip.y - index_finger_tip.y) ** 2) ** 0.5
    
    if distance < 0.05:
        return "rock"
    elif distance < 0.07:
        return "paper"
    else:
        return "scissors"

def gesture_processing_loop(results, choices):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            gesture = get_hand_gesture(hand_landmarks)
            return gesture
    return None

def main():
    cap = cv2.VideoCapture(0)
    choices = ["rock", "paper", "scissors"]
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()

    gesture = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if gesture is None:
            gesture = gesture_processing_loop(results, choices)
        
        if gesture:
            computer_choice = random.choice(choices)
            winner = get_winner(gesture, computer_choice)
            print(f"Your choice: {gesture}")
            print(f"Computer's choice: {computer_choice}")
            print(winner)
            speak(winner)
            
            gesture = None  # Resets gesture
            
            time.sleep(2)  #pause before next gesture

        cv2.imshow('Rock Paper Scissors', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Your choice: rock
Computer's choice: scissors
You win!
Your choice: paper
Computer's choice: paper
It's a tie!
Your choice: rock
Computer's choice: paper
Computer wins!
